In [4]:
import numpy as np
import pandas as pd
import zipfile
import gc
gc.enable()

In [5]:
train = "order_products__train"
products = 'products'
departments = 'departments'
aisles = 'aisles'
prior= "order_products__prior"
orders='orders'
path="/content/drive/MyDrive/Data_btp2/"
orders = zipfile.ZipFile(path+orders+".zip","r")
products = zipfile.ZipFile(path+products+".zip","r")
departments = zipfile.ZipFile(path+departments+".zip","r")
aisles = zipfile.ZipFile(path+aisles+".zip","r")
train = zipfile.ZipFile(path+train+".zip","r")
prior=zipfile.ZipFile(path+prior+".zip","r")

In [6]:
train = pd.read_csv(train.open('order_products__train.csv'))
prior =pd.read_csv(prior.open('order_products__prior.csv'))
products = pd.read_csv(products.open('products.csv'))
departments = pd.read_csv(departments.open('departments.csv'))
aisles = pd.read_csv(aisles.open('aisles.csv'))
orders = pd.read_csv(orders.open('orders.csv'))

In [7]:
orders.eval_set=orders.eval_set.replace({'prior':0, 'train':1,'test':2})

orders.days_since_prior_order=orders.days_since_prior_order.fillna(30)

In [8]:
prior_orders=pd.merge(orders,prior,on='order_id',how='inner')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,0,1,2,8,30.0,196,1,0
1,2539329,1,0,1,2,8,30.0,14084,2,0
2,2539329,1,0,1,2,8,30.0,12427,3,0
3,2539329,1,0,1,2,8,30.0,26088,4,0
4,2539329,1,0,1,2,8,30.0,26405,5,0


In [9]:
del prior
gc.collect()

467

In [11]:
users=prior_orders.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [12]:
total_products_per_order=prior_orders.groupby(by=['user_id','order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()

In [15]:
avg_products=total_products_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [16]:
del[total_products_per_order]
gc.collect()

398

In [17]:
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [18]:
from scipy import stats
dow=prior_orders.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).to_frame('dow_u_most_orders')

dow=dow.reset_index()
dow.head()


,user_id,dow_u_most_orders
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [21]:
hod=prior_orders.groupby(by=['user_id'])['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).to_frame('hod_u_most_orders')

hod=hod.reset_index()
hod.head()

,user_id,hod_u_most_orders
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [22]:
reorder_u=prior_orders.groupby(by='user_id')['reordered'].aggregate('mean').to_frame('u_reorder_ratio').reset_index()

reorder_u['u_reorder_ratio']=reorder_u['u_reorder_ratio'].astype(np.float16)

reorder_u.head()

,user_id,u_reorder_ratio
0,1,0.694824
1,2,0.476807
2,3,0.625000
3,4,0.055542
4,5,0.378418


In [24]:
prior_orders.days_since_prior_order.fillna(0,inplace=True)

In [25]:
avg_days=prior_orders.groupby(by='user_id')['days_since_prior_order'].aggregate('mean').to_frame('average_days_between_orders')

avg_days=avg_days.reset_index()
avg_days.head()

,user_id,average_days_between_orders
0,1,21.084746
1,2,16.902564
2,3,13.590909
3,4,18.611111
4,5,19.108108


In [26]:
total_item=prior_orders.groupby(by='user_id').size().to_frame('u_total_items_bought').astype(np.int16)

total_item.head()


,u_total_items_bought
user_id,
1,59
2,195
3,88
4,18
5,37


In [27]:
users=users.merge(avg_products,on='user_id',how='left')
users=users.merge(dow,on='user_id',how='left')
users=users.merge(hod,on='user_id',how='left')
users=users.merge(reorder_u,on='user_id',how='left')
users=users.merge(avg_days,on='user_id',how='left')
users=users.merge(total_item,on='user_id',how='left')

users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought
0,1,10,5.900000,4,7,0.694824,21.084746,59
1,2,14,13.928571,2,9,0.476807,16.902564,195
2,3,12,7.333333,0,16,0.625000,13.590909,88
3,4,5,3.600000,4,15,0.055542,18.611111,18
4,5,4,9.250000,3,18,0.378418,19.108108,37


In [28]:
del [reorder_u,dow,hod,avg_products,avg_days,total_item]
gc.collect()

559

In [29]:
prd=prior_orders.groupby(by='product_id')['order_id'].aggregate('count').to_frame('p_num_of_times').reset_index()

prd.head()

,product_id,p_num_of_times
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [30]:
reorder_p=prior_orders.groupby(by='product_id')['reordered'].aggregate('mean').to_frame('p_reorder_ratio').reset_index()

reorder_p.head()

,product_id,p_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [31]:
add_to_cart=prior_orders.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').to_frame('p_avg_cart_position').reset_index()


add_to_cart.head()

,product_id,p_avg_cart_position
0,1,5.801836
1,2,9.888889
2,3,6.415162
3,4,9.507599
4,5,6.466667


In [32]:
prd=prd.merge(reorder_p,on='product_id',how='left')
prd=prd.merge(add_to_cart,on='product_id',how='left')


In [33]:
del [reorder_p,add_to_cart]
gc.collect()

298

In [34]:
prd.head()

,product_id,p_num_of_times,p_reorder_ratio,p_avg_cart_position
0,1,1852,0.613391,5.801836
1,2,90,0.133333,9.888889
2,3,277,0.732852,6.415162
3,4,329,0.446809,9.507599
4,5,15,0.600000,6.466667


In [35]:
uxp=prior_orders.groupby(by=['user_id','product_id'])['order_id'].aggregate('count').to_frame('uxp_times_bought')

uxp=uxp.reset_index()
uxp=uxp.head()

In [38]:
times=prior_orders.groupby(by=['user_id','product_id'])['order_id'].aggregate('count').to_frame('times_bought')
times=times.reset_index()
times.head()

,user_id,product_id,times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3
